In [7]:
import re
import numpy as np
import pandas as pd
from scipy import linalg, spatial
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, SparsePCA, TruncatedSVD
from sklearn.feature_extraction.text import (CountVectorizer, TfidfTransformer, TfidfVectorizer)
from sklearn.cluster import KMeans

from sklearn.utils.extmath import randomized_svd

In [8]:
import csv

reviews=[]
for row in csv.reader(open('C:\\Users\\Hassan\\Desktop\\bbcurdu.csv', encoding = "utf-8")):
    reviews.append(row)
texts = [row[0] for row in reviews]
print(texts)

['\ufeff"نیشنل اسٹیڈیم کراچی میں ہونے والے میچ میں کراچی کنگز نے ملتان سلطانز کے خلاف ٹاس جیت کر بالنگ کا فیصلہ کیا ہے۔ یہ میچ جیتنے والی ٹیم فائنل میں پہنچ جائے گی اور ہارنے والی ٹیم لاہور یا پشاور کی فاتح ٹیم کے ساتھ کھیلے گی۔ملتان سلطانز کی ٹیم میں شان مسعود (کپتان)، ذیشان اشرف، ایڈم لیتھ، ریلے روسو، روی بوپارہ، خوشدل شاہ، شاہد آفریدی، سہیل تنویر، محمد الیاس، عمران طاہر اور محمد عرفان شامل ہیں۔کراچی کنگز کی ٹیم میں عماد وسیم (کپتان)، بابر اعظم، شرجیل خان، الیکس ہیلز، وین پارنیل، چیدوک والٹن، محمد عامر، وقاص مقصود، ردرفورڈ، افتخار احمد اور ارشد اقبال شامل ہیں۔"', 'کراچی پولیس کے ترجمان کی جانب سے جاری اعلامیے کے مطابق نیشنل اسٹیڈیم سمیت تمام متعلقہ روٹس اور ہوٹلوں پر پولیس کی بھاری نفری اور کمانڈوزکھلاڑیوں کی سکیورٹی کے لیے موجود رہیں گے، کراچی پولیس کے 24 سینئر افسران سمیت 59 ڈی ایس پیز، 474 نان گزیٹڈ افسران،4275 ہیڈکانسٹیبل ، کانسٹیبل ، ریپڈ رسپانس فورس کے 345 جوان اور اسپیشل سیکیورٹی یونٹ کے 1018 کمانڈوز تعینات کیے گئے ہیں۔نیشنل اسٹیڈیم کراچی اور ہوٹلوں پر اسپیشل برانچ کے سادہ لبا

In [9]:
import spacy
from spacy.lang.ur import Urdu
parser = Urdu()
from spacy.lang.ur.stop_words import STOP_WORDS
import nltk
stopwords = set(nltk.corpus.stopwords.words('urdu'))
#nltk.download('wordnet')
from nltk.corpus import wordnet as wn
import lemma

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.text.isdigit():
            lda_tokens.append('NUM')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def get_lemma(word):
    word = lemma.lemmatize(word)
    return word

final=[]     #In this list all of our pre processed data will be stored
#for line in reviews:  #Break sentence into list words
for words in texts:
    tokens = tokenize(words)
    tokens = [token for token in tokens if len(token) > 3]
    tokens = [token for token in tokens if token not in stopwords]
    tokens = [get_lemma(token) for token in tokens]
    final.append(' '.join(tokens))
print(final)

['\ufeff"نیشنل اسٹیڈیم کراچی کراچی کنگز ملتان سلطانز خلاف بالنگ فیصلہ جیتنے فائنل پہنچ ہارنا لاہور پشاور فاتح کھیلا گی۔ملتان سلطانز مسعود کپتان ذیشان اشرف ایڈم لیتھ ریلا روسو بوپارہ خوشدل شاہد آفرید سہیل تنویر الیاس عمران طاہر عرفان شامل ہیں۔کراچ کنگز عماد وسیم کپتان بابر اعظم شرجیل الیکس ہیلز پارنیل چیدوک والٹن عامر وقاص مقصود ردرفورڈ افتخار احمد ارشد اقبال شامل', 'کراچی پولیس ترجمان جانب جاری اعلامیے مطابق نیشنل اسٹیڈیم سمیت متعلقہ روٹس ہوٹل پولیس بھاری نفری کمانڈوزکھلاڑ سکیورٹی موجود کراچی پولیس سینئر افسران سمیت گزیٹڈ افسران،4275 ہیڈکانسٹیبل کانسٹیبل ریپڈ رسپانس فورس جوان اسپیشل سیکیورٹی یونٹ کمانڈوز تعین ہیں۔نیشنل اسٹیڈیم کراچی ہوٹل اسپیشل برانچ سادہ لباس اہلکار تعداد سکیورٹی فرائض انجام معمول حالات نمٹنے اسپیشل سیکیورٹی یونٹ swat ٹیم فوری رسپانس تیار گی۔کراچ پولیس ترجمان ٹریفک پولیس فائیو شرکت آنا ملک ملک کھلاڑی نیشنل اسٹیڈیم آمدورفت اوق شہر سہولت متبادل پلان جاری کراچی بحال پولیس قربان انٹرنیشنل کھلاڑی آمد امان قیام دلیل کراچی پولیس عوام تحفظ قانون بالادست ممکن اقدام', 'مارچ 

In [10]:
vectorizer = CountVectorizer()

counts_matrix = vectorizer.fit_transform(final)

feature_names = vectorizer.get_feature_names()

count_matrix_df = pd.DataFrame(counts_matrix.toarray(), columns=feature_names)

print("Word frequency matrix: \n", count_matrix_df)

Word frequency matrix: 
     101رنز  10م  14  24گیند  2ٹیم  345رنز  3غیرملک  3پچز  4275  4شکار  ...  \
0        0    0   0       0     0       0        0     0     0      0  ...   
1        0    0   0       0     0       0        0     0     1      0  ...   
2        0    0   0       0     1       0        0     0     0      0  ...   
3        0    0   0       0     0       0        1     0     0      0  ...   
4        1    0   0       0     0       1        0     0     0      1  ...   
5        0    0   0       0     0       0        0     0     0      0  ...   
6        0    0   1       0     0       0        0     0     0      0  ...   
7        0    0   0       0     0       0        0     0     0      0  ...   
8        0    0   0       0     0       0        0     1     0      0  ...   
9        0    1   0       0     0       0        0     0     0      0  ...   
10       0    0   0       1     0       0        0     0     0      0  ...   
11       0    0   0       0     0      

In [18]:
vectorizer = TfidfVectorizer(stop_words=stopwords,max_features=10000, max_df = 0.5,
                                    use_idf = True,
                                    ngram_range=(1,3))

X = vectorizer.fit_transform(final)
print(X.shape)
print(feature_names)

num_clusters = 4

km = KMeans(n_clusters=num_clusters)
km.fit(X)
    
clusters = km.labels_.tolist()
print(clusters)

(30, 5965)
['101رنز', '10م', '14', '24گیند', '2ٹیم', '345رنز', '3غیرملک', '3پچز', '4275', '4شکار', '4میچز', '4وکٹ', '5رنز', '5لاکھ', '5م', '5میچز', '7شکار', '8فتوح', '8کیچز', 'swat', 'آفرید', 'آواز', 'ئل', 'ئند', 'ئی', 'ئیز', 'ئیں', 'اب', 'ابتدا', 'ابتدائ', 'ابوظہب', 'اتن', 'اتنا', 'اتوار', 'اثر', 'اجازت', 'اجاڑ', 'اجزا', 'اجلاس', 'احد', 'احمد', 'اخبار', 'اختر', 'اخترنا', 'اختیار', 'ادائیگ', 'ادار', 'ادارا', 'ادارہ', 'اداکار', 'اداکارہ', 'اداکرنا', 'اداکیا', 'ادویہ', 'اردو', 'ارشد', 'ارطغرل', 'اریج', 'از', 'ازاں', 'ازخود', 'ازدواج', 'اس', 'استاد', 'اسری', 'اسفنج', 'اسمارٹ', 'اسمبلی', 'اسٹار', 'اسٹاف', 'اسٹریمنگ', 'اسٹوری', 'اسٹوریج', 'اسٹیڈیم', 'اسٹیڈیمز', 'اسپائیڈر', 'اسپتال', 'اسپورٹس', 'اسپورٹسم', 'اسپیئرز', 'اسپیس', 'اسپیشل', 'اسپیکر', 'اسکائی', 'اسکرین', 'اسکواڈ', 'اسکوبی', 'اسکور', 'اسکورر', 'اشتہار', 'اشرف', 'اضاف', 'اضافہ', 'اطراف', 'اظہار', 'اعتماد', 'اعظم', 'اعلامیے', 'اعلی', 'افتخار', 'افراد', 'افریقی', 'افزائی', 'افسران', 'اقبال', 'اقدام', 'اقرا', 'اقساط', 'الاقوام', 'الوطن

In [21]:
U, Sigma, VT = randomized_svd(X, n_components=4, n_iter=1000, random_state=122)

svd_model = TruncatedSVD(n_components=4, algorithm='randomized', n_iter=1000, random_state=122)

svd_model.fit(X)
    
print(U.shape)

for i, comp in enumerate(VT):
    terms_comp = zip(feature_names, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Cluster "+str(i)+": ")
    for t in sorted_terms:
        print(t)
    print(" ")

(30, 4)
Cluster 0: 
('سپرد', 0.10390276724026704)
('بیٹے', 0.083995056838197)
('متعلق', 0.07105807805230602)
('حساس', 0.0586375020248582)
('متعلقہ', 0.0586375020248582)
('چینل', 0.05743733452223475)
('ضرور', 0.05548659580404362)
('شعبہ', 0.05364906998299643)
('پور', 0.050405607921643884)
('دید', 0.05025713390362985)
 
Cluster 1: 
('ایلیمنیٹرز', 0.10774996962478246)
('پیغام', 0.10765676120776956)
('درمیان', 0.06765288528256316)
('درپیش', 0.06765288528256316)
('کردہ', 0.06482850091177254)
('بستر', 0.05069514926677946)
('بطور', 0.05069514926677946)
('ٹریفک', 0.05069514926677946)
('مد', 0.04127613158826158)
('ایونٹ', 0.04085297059179937)
 
Cluster 2: 
('پیغام', 0.1419799922552291)
('سرور', 0.07656546468810167)
('کریں', 0.07373095719641207)
('تشکیل', 0.0694178650993264)
('بہترپرفارمنس', 0.06538471354871778)
('بھارت', 0.05673977759680887)
('ٹیبل', 0.04593927881286098)
('وہ', 0.04140568918754536)
('بھاری', 0.04015765723211953)
('بھدا', 0.04015765723211953)
 
Cluster 3: 
('تشکیل', 0.1609879079